In [1]:
# %pip install torch torchvision torchaudio

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50

In [3]:
INPUT_DIR_A = '../data/model_input_a'

In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the batch size
batch_size = 1

# Define the transformation
transform = transforms.Compose([
    transforms.RandomPerspective(),  # Apply a random perspective transformation
    transforms.ToTensor()  # Convert the image to a tensor
])

# Define the dataset
dataset = datasets.ImageFolder(INPUT_DIR_A, transform=transform)

# Verify the classes
classes = dataset.classes
print(classes)

dataset = DataLoader(dataset, batch_size=batch_size, shuffle=True)


['Monet', 'Picasso']


In [5]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Device: {device}")

# Load the pre-trained ResNet model
model = resnet50(weights='ResNet50_Weights.DEFAULT')

# Freeze all the parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Define the number of training epochs
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Set the model to training mode
    model.train()

    
    for images, labels in dataset:
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update the running loss
        running_loss += loss.item()
        
        # Compute the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Compute the accuracy
    accuracy = 100 * correct / total
    
    # Print the loss and accuracy for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}: Loss = {running_loss:.4f}, Accuracy = {accuracy:.2f}%")
    if accuracy > 90:
        break


Device: cpu
Epoch 1/100: Loss = 13.7646, Accuracy = 36.84%
Epoch 2/100: Loss = 13.1225, Accuracy = 52.63%
Epoch 3/100: Loss = 13.2057, Accuracy = 52.63%
Epoch 4/100: Loss = 11.7194, Accuracy = 89.47%
Epoch 5/100: Loss = 11.2174, Accuracy = 63.16%
Epoch 6/100: Loss = 11.0811, Accuracy = 78.95%
Epoch 7/100: Loss = 11.0461, Accuracy = 68.42%
Epoch 8/100: Loss = 10.6800, Accuracy = 78.95%
Epoch 9/100: Loss = 10.9498, Accuracy = 84.21%
Epoch 10/100: Loss = 9.4382, Accuracy = 94.74%
